<a href="https://colab.research.google.com/github/Saikiran4545/DataScience_Project/blob/main/trading_24hr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import ccxt
from dotenv import load_dotenv

class TradingBot:
    def __init__(self, initial_budget, symbol):
        print("Creating TradingBot instance...")
        self.budget = initial_budget
        self.symbol = symbol
        self.load_api_credentials()

        # Explicitly set the API key, secret, and passphrase
        self.kucoin_exchange = ccxt.kucoin({
            'apiKey': os.environ.get('API_KEY'),
            'secret': os.environ.get('API_SECRET'),
            'password': os.environ.get('API_PASSPHRASE'),
            'enableRateLimit': True,  # You may need to set this option
        })

    def load_api_credentials(self):
        # Load environment variables from .env file
        load_dotenv()

    def get_24hr_high(self):
        try:
            ticker = self.kucoin_exchange.fetch_ticker(self.symbol)
            high_24hr = ticker['high']
            print(f"24-hour High for {self.symbol}: {high_24hr}")
            return high_24hr
        except ccxt.NetworkError as e:
            print(f"Network error fetching 24-hour high for {self.symbol}: {e}")
            return None
        except ccxt.ExchangeError as e:
            print(f"Exchange error fetching 24-hour high for {self.symbol}: {e}")
            return None
        except Exception as e:
            print(f"Error fetching 24-hour high for {self.symbol}: {e}")
            return None

    def place_order(self, order_type, price, quantity):
        try:
            if order_type == "Buy":
                order = self.kucoin_exchange.create_limit_buy_order(self.symbol, quantity, price)
            elif order_type == "Sell":
                order = self.kucoin_exchange.create_limit_sell_order(self.symbol, quantity, price)

            order_id = order['id']
            print(f"{order_type} order placed - Order ID: {order_id}")
            return order_id
        except ccxt.NetworkError as e:
            print(f"Network error during order placement: {e}")
        except ccxt.ExchangeError as e:
            print(f"Exchange error during order placement: {e}")
        except Exception as e:
            print(f"Error during order placement: {e}")
        return None

    def cancel_order(self, order_id):
        try:
            self.kucoin_exchange.cancel_order(order_id)
            print(f"Order canceled - Order ID: {order_id}")
        except ccxt.NetworkError as e:
            print(f"Network error during order cancellation: {e}")
        except ccxt.ExchangeError as e:
            print(f"Exchange error during order cancellation: {e}")
        except Exception as e:
            print(f"Error during order cancellation: {e}")

    def execute_strategy(self):
        print("Executing strategy...")
        try:
            high_24hr = self.get_24hr_high()

            if high_24hr is not None:
                buy_price_threshold = 0.93 * high_24hr  # Buy below 5% of the 24-hour high
                sell_price_target = 1.03 * buy_price_threshold  # Sell at 5% profit

                print(f"Buy Price Threshold: {buy_price_threshold}, Sell Price Target: {sell_price_target}")

                # Check if there's an existing buy order
                open_orders = self.kucoin_exchange.fetch_open_orders(self.symbol)
                existing_buy_order = next((order for order in open_orders if order['side'] == 'buy'), None)

                if existing_buy_order:
                    # Check if the 24-hour change is not favorable, then cancel the buy order
                    if high_24hr < self.get_24hr_high():
                        self.cancel_order(existing_buy_order['id'])
                        print("Buy order canceled due to unfavorable 24-hour change.")
                    else:
                        print("Buy order already exists.")
                else:
                    # Place a new buy order
                    buy_quantity = self.budget / buy_price_threshold
                    order_id = self.place_order("Buy", buy_price_threshold, buy_quantity)

                    if order_id:
                        # Place a corresponding sell order
                        sell_quantity = buy_quantity
                        self.place_order("Sell", sell_price_target, sell_quantity)
                        print("Buy and sell orders placed.")

        except Exception as e:
            print(f"Error in executing strategy: {e}")

    def run(self):
        print("Running TradingBot...")
        try:
            while True:
                self.execute_strategy()

        except KeyboardInterrupt:
            print("TradingBot stopped.")

# Example usage:
symbol = 'SOL-USDT'
initial_budget = 10.00

bot = TradingBot(initial_budget, symbol)
bot.run()
